# Example calculation using the RVFC

The following is an example of calculating the number of RV measurments (and observing time) required to detect the RV semi-amplitude of a user-defined transiting planet at a desired level of significance and with a user-defined spectrograph using the [Radial Velocity Follow-up Calculator](https://arxiv.org/abs/1807.01263). 

In [1]:
from RVFollowupCalculator import *
from setupRVFCinput import *
%matplotlib inline

In [2]:
# setup spectrograph input file for NIRPS
prefix = 'myexample'
min_wl = 980     # lower edge of the spectrograph's spectral range in nm
max_wl = 1800    # upper edge of the spectrograph's spectral range in nm
R = 1e5          # spectral resolution lambda / delta lambda
aperture = 3.6   # effective telescope aperture diameter in meters
throughput = .1  # spectrally-averaged instrument throughput (in [0,1])  
RVfloor = 1      # RV noise floor in m/s
maxtell = .02    # maximum fractional telluric absorption (i.e. mask where atmospheric transmittance is < 1-maxtell)
overhead = 5     # overhead to be added to the total observing time in minutes
theta = min_wl, max_wl, R, aperture, throughput, RVfloor, maxtell, overhead
fname_spec = setup_spectrograph(prefix, *theta)

In [3]:
# setup stellar input file (example is GJ 1132)
mag = 9.245  # stellar magnitude in either the V or J bands for optical and near-IR spectrographs respectively
Ms = .181    # stellar mass in solar masses
Rs = .207    # stellar radius in solar radii
Teff = 3270  # effective temperature in Kelvin
Z = -.12     # log(Fe/H) metallicity in solar units
vsini = .1   # projected stellar rotation velocity in km/s
prot = 122   # stellar rotation period in days
theta = mag, Ms, Rs, Teff, Z, vsini, prot
fname_star = setup_star(prefix, *theta)

In [4]:
# setup planet input file (example is GJ 1132 b)
P = 1.629   # orbital period in days
rp = 1.16   # planet radius in Earth radii
mp = 1.62   # planet mass in Easrth masses. If unspecified, mp is estimated from rp and the Weiss+Marcy 2014 relation
theta = P, rp
fname_planet = setup_planet(prefix, *theta, mp=mp)

In [5]:
# setup RV noise input file
texp = 600      # exposure time in seconds
sigphot = None  # RV photon noise in m/s. Is computed from spectrograph and stellar parameters if unspecified
sigact = 3      # RV residual rms from stellar activity in m/s. Can be zero.
sigplanets = .5 # RV residual rms from additional planets in the system in m/s. Should be zero if no additional planets or additional planet are modelled exactly (<- not really possible)
sigeff = None   # effective RV uncertainty in m/s. If specified it by-passes all other RV noise sources
fname_sigRV = setup_sigRV(prefix, texp, sigact=sigact, sigplanets=sigplanets)

## Compute the number of RV measurements and observing time using the above input parameters 

In [6]:
# setup last parameters
Kdetsig = 3   # desired RV semi-amplitude detection significance (i.e. how many sigma?)
NGPtrials = 0 # number of calculations to conduct using the GP framework (will take longer). Set to zero for white noise. 

In [7]:
# run the RVFC
runGP = NGPtrials > 0
fname_out = '%s_output'%prefix
kwargs = {'input_planet_fname':fname_planet, 'input_star_fname':fname_star, 'input_spectrograph_fname':fname_spec,
         'input_sigRV_fname':fname_sigRV, 'output_fname':fname_out, 'runGP':runGP, 'NGPtrials':int(NGPtrials)}
self = nRV_calculator(Kdetsig, verbose_results=True, **kwargs)

Computing the photon-noise limited RV precision...

Convolving the Y band stellar spectrum to the instrumental resolution (R = 100000)
Masking Y band telluric regions where transmittance < 0.98 percent
Took 4.8 seconds

Convolving the J band stellar spectrum to the instrumental resolution (R = 100000)
Masking J band telluric regions where transmittance < 0.98 percent
Took 8.4 seconds

Convolving the H band stellar spectrum to the instrumental resolution (R = 100000)
Masking H band telluric regions where transmittance < 0.98 percent
Took 27.7 seconds

Computing nRVs...

##################################################
#	Planet parameters:
# P  = 1.629 days
# rp = 1.16 REarth
# mp = 1.62 MEarth
# K  = 2.78 m/s

#	Stellar parameters:
# mags  = 8.942, 9.245, 8.639 (YJH)
# Ms    = 0.18 MSun
# Rs    = 0.21 RSun
# Teff  = 3270 K
# vsini = 0.1 km/s

#	Spectrograph parameters:
# R           = 100000
# Aperture    = 3.6 m
# Throughput  = 0.10
# Noise floor = 1.00 m/s

#	RV noise parameters:
# 

In [8]:
# (optionally) remove the input files we created to keep things clean
clean = False
if clean: clean_files(prefix)

## How to directly access the results via the RVFC object (i.e. self)

In [9]:
# you can reload your saved RVFC object via
self = loadpickle(fname_out)

In [10]:
# all input and output parameters are given as attributes of the RVFC object
# for example, let's see what our desired K detection significance is
print('K = %.3f m/s'%self.K)
print('Desired K uncertainty = %.3f m/s (%.1f sigma)'%(self.sigK_target, self.K/self.sigK_target))

# similarly lets see the number of RV measurements from the white noise calculations
print('Number of RV measurements = %.2f'%self.nRV)

# similarly what's the total observing time from the white noise calculations
print('Total observing time = %.3f hours (%.3f nights)'%(self.tobs, self.tobs/7.))

# and so...

K = 2.785 m/s
Desired K uncertainty = 0.928 m/s (3.0 sigma)
Number of RV measurements = 33.25
Total observing time = 8.314 hours (1.188 nights)
